# 05. Функторы, Моноиды

<div style="text-align: right"> 
    
    Лекция 5.1 05/03/21 
    
    Кубенский Александр Александрович
    
    akoubensky@gmail.com
</div> 

## Класс типов `Functor`

Приступим к рассмотрению класса типов `Functor`, предназначенного для типов, **которые могут быть отображены друг в друга.**

In [ ]:
class Functor f
        where fmap :: (a -> b) -> f a -> f b

```haskell
fmap :: (a -> b) -> f a -> f b
```

Что-то мне напоминает объявление функции fmap - да, похожа на `map`:

```haskell
map :: (a -> b) -> [a] -> [b]
```

Функция `map` берёт функцию из `a` в `b` и список элементов типа `а` и возвращает список элементов типа `b`. Фактически функция `map` – это 
функция `fmap`, которая работает только на списках. Вот как список 
сделан экземпляром класса Functor:

In [ ]:
instance Functor [] where
fmap = map

Из определния функции видим, что параметр `f` должен быть конструктором типов, принимающим один тип.

Так как для списков функция `fmap` – это просто `map`, то мы получим одинаковые результаты при их использовании на списках:

In [3]:
fmap (*2) [1..3]
map (*2) [1..3]

[2,4,6]

[2,4,6]

Что случится, если применить функцию map или fmap к пустому 
списку? Мы получим опять же пустой список. Но функция `fmap`
преобразует пустой список типа `[a]` в пустой список типа `[b]`.


### Экземпляр класса Functor для типа Maybe

А что ещё умеет быть контейнером для других типов? Например, тип `Maybe`. Он может быть  «пустой коробкой», и в этом случае имеет значение `Nothing`, или же  в нём хранится какое-то одно значение, например `"ХА-ХА"`, и тогда он равен `Just "ХА-ХА"`.

Вот как тип `Maybe` сделан функтором:

```haskell
instance Functor Maybe where
fmap f (Just x) = Just (f x)
fmap f Nothing = Nothing

```

Функтор принимает  конструктор типа с одним параметром, не конкретный тип. Если  вы мысленно замените параметр `f` на `Maybe`, функция `fmap` работает  как `(a –> b) –> Maybe a –> Maybe b`, только для типа `Maybe`, что вполне себя оправдывает

Если значение типа Maybe – это `Nothing`, *возвращается Nothing*. Если 
мы отображаем «пустую коробку», мы получим «пустую коробку», 
что логично. Точно так же функция `map` для пустого списка возвратит пустой список.

Если это не пустое значение, а некоторое значение, упакованное в конструктор `Just`, то мы применяем функцию к содержимому `Just`:

In [5]:
fmap (++ " HI, I AM INSIDE JUST") (Just "FUNNY JOKE.")
fmap (++ " HI, I AM INSIDEИ JUST") Nothing
fmap (*2) (Just 200)
fmap (*2) Nothing

Just "FUNNY JOKE. HI, I AM INSIDE JUST"

Nothing

Just 400

Nothing

## Функторы - теория

Для того, чтобы функции, заявленные в классе, имели бы один и тот же смысл для всех типов, которые определены как «экземпляры» этого класса, следует описать «контракт» – набор правил, которым должны удовлетворять функции.

Такой контракт определяется неформально, то есть он никак не проверяется (да и не может быть проверен) системой. Описание контракта обычно описывается в виде уравнений, которым должны удовлетворять функции класса.

Контракт для класса Functor:

```haskell
fmap id         == id
fmap (f . g)    == fmap f . fmap g
```

Конструктор `((->) a)` задает тип функций с заданным типом аргумента. Функцию можно рассматривать как «структуру», если считать, что функция – это некоторая «таблица». Тогда аргумент f функции fmap может применяться к «столбцу значений» таблицы, то есть к результатам работы функции. Отсюда:

```haskell
instance Functor ((->) a) where fmap f g == f . g
```


Контракт класса Functor соблюдён для типа `((->) a)`!

## Класс Моноиды

В алгебре моноид – это множество объектов с определенной на нем ассоциативной бинарной операцией и нейтральным элементом.

**Опр**: В Haskell **Monoid** – это класс, в котором определен *нейтральный элемент* (константа) и *бинарная операция*, ассоциативность которой определена контрактом.

```haskell
class Monoid a where
mempty  :: a
mappend :: a -> a -> a
mconcat :: [a] -> a
mconcat = foldr mappend empty
```

Для функции mappend определен также синоним – оператор (`<>`).


Контракт для моноида абсолютно естествен:

```haskell
mempty <> v    ==  v --проверяем, что элемент действительный нейтральный
v <> mempty    ==  v --проверяем, что элемент действительный нейтральный
(u <> v) <> w  ==  u <> (v <> w) -- поверка операции ассоциативности
```

Легко определить экземпляр типа списка для моноида:

```haskell
instance Monoid [] where
mempty  =  [] --нейтральный элемент для списка
mappend =  (++) -- конкатенация списков
```

## Моноиды для числовых типов

Гораздо сложнее определить моноидные операции для числовых типов, 

```haskell
instance Num a => Monoid a where
mempty  =  0
mappend =  (+)
```

Это определение работает, только непонятно, почему не

```haskell
instance Num a => Monoid a where
mempty  =  1
mappend =  (*)
```

Сделаем следующие определения типов:

```haskell
newtype Sum a = Sum { getSum :: a } deriving (Eq, Ord, Show)
newtype Product a = Product {getProduct :: a } deriving (Eq, Ord, Show)
```

Теперь можем определить экземпляры Моноида для двух разных типов - `Sum` и `Product`.

Над значениями этих типов также определены все арифметические операции, поскольку имеются определения:

```haskell
instance Num a => Sum a where ...
instance Num a => Product a where ...
```

Теперь можно определить

```haskell
instance Num a => Monoid (Sum a) where
mempty  =  Sum 0 -- нейтральный элемент для суммы
mappend =  (+)

instance Num a => Monoid (Product a) where
mempty  =  Product 1 -- нейтральный элемент для произведения
mappend =  (*)
```

## Свёртки структур данных

Для списков определены операции свёртки `foldl` и `foldr`, которые позволяют «сворачивать» список с помощью любой бинарной операции (не обязательно ассоциативной). 

Но если бинарная операция **ассоциативна**, то возникает новая возможность выполнять свёртку «параллельно» с разными частями списка - логично.

В качестве начального значения берётся нейтральный элемент.

Такой «фокус» можно проделывать с любой структурой, содержащей упорядоченный набор элементов. Поэтому определён класс `Foldable` **сворачиваемых структур**.

В этом классе определены многие функции, которые раньше определялись нами только для списков:
- `foldr`
- `sum`
- `maximum`
- `null`
- `length`
- `elem`

Все эти функции могут быть *определены* через **основную функцию** `foldMap`; ещё одна функция, определяемая через `foldMap` – это `fold`

```haskell
class Foldable t where
foldMap :: Monoid m => (a -> m) -> t a -> m -- основная функция, говорит о том, как применять Моноидные операции. Может быть определена, если есть Monoid
fold    :: Monoid m => t m -> m -- элементы структуры являются моноидами, тождественная функция
fold s  =  foldMap id s
```

Для списков свёртка выглядит следующим образом:

```haskell
instance Foldable [] where
foldMap f list  =  foldr (<>) mempty (map f list)
```

Дерево является **сворачиваемой** структурой.

## Аппликативные функторы

Поговорим про другие классы и закончим дело определением важного класса `Монада`. Определим класс **Аппликативных функторов**.

```haskell
class Functor f => Applicative f where
pure   :: a -> f a -- поднимает значение в структуру
(<*>)  :: f (a -> b) -> f a -> f b -- первая структура содержит функцию, а вторая структура содержит элементы
```

Функтор - класс, в котором определена операция отображения. 

Класс Functor содержит функцию fmap, с помощью которой можно применять одну функцию к элементам некоторой `структуры`. Если функций несколько, причем они находятся внутри такой же структуры, то можно попробовать **построить структуру из всевозможных применений всех функций к элементам базовой структуры.**

Может оказаться, что функция находится внутри структуры данных. Нельзя ли применить данную функцию и сделать всевозможные операции. Тогда есть смысл применять эти функции ко всем другим структурам.

Функция `pur`e обычно используется для того, чтобы **«поднять»** функцию или операцию в «структуру». 

Функция `(<*>)` применяет такую операцию ко всем элементам структуры так, как это делает **fmap, получая новую структуру.**

Удвоение всех элементов числового списка можно выполнить, применив `fmap`, но можно сделать это и с помощью операций **pure** и `(<*>)`.

In [17]:
fmap (*2) [3,7,9,1]

[6,14,18,2]

In [18]:
pure (*2) <*> [3,7,9,1] -- операцию умножения поднимаем до уровня структуры данных
-- образуя список из единственной функции, а затем применяем операцию 
-- из класса Applicative

Line 1: Use <$>
Found:
pure (* 2) <*> [3, 7, 9, 1]
Why not:
(* 2) <$> [3, 7, 9, 1]

[6,14,18,2]

Но, конечно, более интересно использовать функции нескольких аргументов для получения списка всевозможных результатов, например:

Пусть есть два списка

```haskell
t1 = [2, 4]
t2 = [3, 4, 5]
```

Поднимем бинарную операцию умножению в структуру предложения. Получаем список из двух функций - функция, умножающая на два и умножающая на 4:

```haskell
let t3 = [*2, *4]
```

Затем мы можем данные функции применять ко всем элементам списка [3,4,5] - можем проталкивать операцию внутрь структуры данных.

In [22]:
pure (*) <*> [2,4] <*> [3..5]

Line 1: Use <$>
Found:
pure (*) <*> [2, 4]
Why not:
((*) <$> [2, 4])

[6,8,10,12,16,20]

Еще один пример:

In [28]:
pure zip3 <*> [[4]] <*> [[4],[5]] <*> [[1],[2]]

Line 1: Use <$>
Found:
pure zip3 <*> [[4]]
Why not:
(zip3 <$> [[4]])

[[(4,4,1)],[(4,4,2)],[(4,5,1)],[(4,5,2)]]

## Контракт для класса Applicative

```haskell
class Functor f => Applicative f where
pure   :: a -> f a
(<*>)  :: f (a -> b) -> f a -> f b

```

Контракт для функций этого класса содержит 4 правила:

 ```haskell 
1. pure id <*> v               =  v -- если тождественную функцию поднимаем в структуру данных, то она остается такой же
2. pure (.) <*> u <*> v <*> w  =  u <*> (v <*> w) -- если поднимаем функцию композиций, то записываем по правилу композиций
3. pure f <*> pure x           =  pure (f x) -- то же самое, неважно в каком порядке

4. x <*> pure y                =  pure ($ y) <*> x -- если есть x и есть структура, в которой мы подняли y, то результат применения == если бы мы подняли операцию применения функции к аргументу y и затем мы бы это использовали для x

$ y -- применение некоторой функци к аргументу y
```

Проверим выполнение этого контракта для:

```haskell
instance Applicative Maybe where
pure x                 =  Just x
(Just f) <*> (Just v)  =  Just (f v) 
_ <*> _                =  Nothing
```

В пакете `Control.Applicative` определены ещё две удобные функции:

```haskell
liftA2 f a b    =  pure f <*> a <*> b
liftA3 f a b c  =  pure f <*> a <*> b <*> c
```

Для функции одного аргумента подобная функция превращается в ещё один `fmap`:

```haskell
liftA :: Applicative f => (a -> b) -> f a -> f b
liftA f a =  pure f <*> a
```

## Монады и последовательные вычисления

Если мы хотим последовательно **применять разные функции** к некоторой структуре данных, то нам придется каждый раз «поднимать» очередную функцию в эту структуру с помощью `liftA` (или `fmap`).

Пусть, например, имеются 3 функции `f1, f2 и f3` типа `a -> Maybe a`. 

Мы хотим последовательно их применить к некоторому значению, но если в какой-то момент получается Nothing, то этот результат протаскивается дальше.

```haskell
(>>=) :: Maybe a -> (a -> Maybe a) -> Maybe a
Nothing  >>= _   = Nothing
(Just x) >>= f   = f x
```

Тогда нужный результат получается так:

```haskell
pure x >>= f1 >>= f2 >>= f3
```

Если обобщить операцию `(>>=)` до произвольных структур, получим **класс `Monad`**:

```haskell
class Monad m where
return  :: a -> m a -- поднятие до значения структуры данных
(>>=)   :: m a -> (a -> m b) -> m b -- выполнение функции
```

Операция `return` *поднимает значение в структуру данных (монаду)* так же, как это делала `pure` в *аппликативных функторах*, а **операция связывания (>>=)** *выполняет* заданную функцию `«под монадой»`.

Класс `Monad` широко используется.

## Стандартные экземпляры и контракт для класса Monad

Контракт дя класса `Monad` включает в себя следующие 4 правила:

```haskell
1. return value >>= f      == f value -- если мы берем значение и упрятываем в монаду а затем связываем f, то это то же самое, что и f к значению - будет Монада
2. m >>= return            == m -- если мы берём некоторую функцию и погружаем её, то получаем Монаду
3. m >>= (\x -> f x >>= h) == (m >>= f) >>= h
4. fmap f xs               == xs >>= return . f   -- для функторов
```

Монада для Maybe

```haskell
instance Monad Maybe where
return v       = Just v
Nothing >>= _  = Nothing
Just v  >>= f  = f v
```

Монада для списка

```haskell
instance Monad [] where
return v       = [v]
list >>= f     = foldr ((++) . f) [] list
```

Монада для `Either`

```haskell
instance Monad (Either e) where
return              = Right
Left  val >>= _     = Left val
Right val >>= f     = f val
```

## Ограниченный калькулятор

Этот «калькулятор» вырабатывает правильный результат, только если он находится в интервале [0..100)

In [30]:
calc :: Integral a => (a -> a) -> a -> Maybe a
calc f x | result >= 0 && result < 100 = Just result
         | otherwise                   = Nothing
         where result = f x


In [31]:
return 15 >>= calc (+10) >>= calc (*5) >>= calc (subtract 45)

Line 1: Monad law, left identity
Found:
return 15 >>= calc (+ 10)
Why not:
calc (+ 10) 15

Nothing

In [32]:
return 15 >>= calc (+10) >>= calc (*3) >>= calc (subtract 45)


Line 1: Monad law, left identity
Found:
return 15 >>= calc (+ 10)
Why not:
calc (+ 10) 15

Just 30

In [33]:
return 15 >>= calc (+10) >>= calc (+3) >>= calc (subtract 45)

Line 1: Monad law, left identity
Found:
return 15 >>= calc (+ 10)
Why not:
calc (+ 10) 15

Nothing